In [ ]:
%pip install -q keras

In [156]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import normalize
from keras.layers import LeakyReLU

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./109550174.ipynb
./109550174_Final - 複製.ipynb
./109550174_Final_inference.ipynb
./109550174_Final_train.ipynb
./final.h5
./final_.h5
./submission.csv
./.git\COMMIT_EDITMSG
./.git\config
./.git\description
./.git\HEAD
./.git\index
./.git\hooks\applypatch-msg.sample
./.git\hooks\commit-msg.sample
./.git\hooks\fsmonitor-watchman.sample
./.git\hooks\post-update.sample
./.git\hooks\pre-applypatch.sample
./.git\hooks\pre-commit.sample
./.git\hooks\pre-merge-commit.sample
./.git\hooks\pre-push.sample
./.git\hooks\pre-rebase.sample
./.git\hooks\pre-receive.sample
./.git\hooks\prepare-commit-msg.sample
./.git\hooks\push-to-checkout.sample
./.git\hooks\update.sample
./.git\info\exclude
./.git\logs\HEAD
./.git\logs\refs\heads\master
./.git\logs\refs\remotes\origin\master
./.git\objects\24\52cdb810b1551c1d57e042f9cdbce646ae2a17
./.git\objects\26\aee2458f78a39d2d848b1f3ae748401f777996
./.git\objects\27\f9fe4a6306df9dd3b5bb8d837ad28ee1296812
./.git\objects\2d\659aef932012d1f1736616ff216d3909f6b19a


In [157]:
pd.set_option('display.max_columns',26)
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler,robust_scale
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import gc

In [158]:
train = pd.read_csv('./tabular-playground-series-aug-2022/train.csv').drop('id',axis=1)
train = train.fillna(train.mean())

C:\Users\meng9\AppData\Local\Temp\ipykernel_2784\3531699451.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train = train.fillna(train.mean())


In [159]:
X = train.drop(['failure', 'product_code', 'measurement_1', 'measurement_11', 'measurement_12', 'measurement_13', 'measurement_16'], axis=1).copy()
y = train['failure'].copy()

In [160]:
from sklearn.preprocessing import LabelEncoder, RobustScaler

le = LabelEncoder()

# training
X['attribute_0'] = le.fit_transform(X['attribute_0'])
X['attribute_1'] = le.fit_transform(X['attribute_1'])

np_train = X.to_numpy()
y = y.astype('int')

In [161]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(18,)),
  tf.keras.layers.Dense(128, activation=LeakyReLU(alpha=0.12)),
  tf.keras.layers.Dense(128, activation=LeakyReLU(alpha=0.12)),
  tf.keras.layers.Dense(128, activation=LeakyReLU(alpha=0.1)),
  tf.keras.layers.Dense(128, activation=LeakyReLU(alpha=0.1)),
  tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
])
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[keras.metrics.AUC(name='auc')])

model.fit(X, y, epochs=30)

model.save_weights('final.h5')



Epoch 1/30
831/831 [==============================] - 4s 3ms/step - loss: 0.7399 - auc: 0.5100
Epoch 2/30
831/831 [==============================] - 3s 3ms/step - loss: 0.5352 - auc: 0.5346
Epoch 3/30
831/831 [==============================] - 3s 3ms/step - loss: 0.5213 - auc: 0.5439
Epoch 4/30
831/831 [==============================] - 3s 3ms/step - loss: 0.5248 - auc: 0.5418
Epoch 5/30
831/831 [==============================] - 3s 3ms/step - loss: 0.5203 - auc: 0.5494
Epoch 6/30
831/831 [==============================] - 3s 3ms/step - loss: 0.5162 - auc: 0.5627
Epoch 7/30
831/831 [==============================] - 3s 3ms/step - loss: 0.5152 - auc: 0.5615
Epoch 8/30
831/831 [==============================] - 3s 3ms/step - loss: 0.5140 - auc: 0.5661
Epoch 9/30
831/831 [==============================] - 3s 3ms/step - loss: 0.5149 - auc: 0.5635
Epoch 10/30
831/831 [==============================] - 3s 3ms/step - loss: 0.5127 - auc: 0.5697
Epoch 11/30
831/831 [============================

In [162]:
model.summary()

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_27 (Flatten)        (None, 18)                0         
                                                                 
 dense_135 (Dense)           (None, 128)               2432      
                                                                 
 dense_136 (Dense)           (None, 128)               16512     
                                                                 
 dense_137 (Dense)           (None, 128)               16512     
                                                                 
 dense_138 (Dense)           (None, 128)               16512     
                                                                 
 dense_139 (Dense)           (None, 1)                 129       
                                                                 
Total params: 52,097
Trainable params: 52,097
Non-tra